In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
import os

In [ ]:
files= os.listdir(r'D:\uber-pickups-in-new-york-city-20210602T094746Z-001\uber-pickups-in-new-york-city')[-9:]
files

In [ ]:
files.remove('other-Prestige_B01338.csv')

In [ ]:
files.remove('other-Dial7_B00887.csv')

In [ ]:
files.remove('uber-raw-data-janjune-15.csv')

In [ ]:
files

In [ ]:
#put all the 6 datasets in one file
path=r'D:\uber-pickups-in-new-york-city-20210602T094746Z-001\uber-pickups-in-new-york-city'
final=pd.DataFrame()
for file in files:
    df=pd.read_csv(path+"/"+file,encoding='utf-8')
    final=pd.concat([df,final])

# Data From 2014 

In [ ]:
final.shape

In [ ]:
df=final.copy()
df.head()

In [ ]:
df.dtypes

In [ ]:
df['Date/Time']=pd.to_datetime(df['Date/Time'],format='%m/%d/%Y %H:%M:%S')

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df['Weekday']=df['Date/Time'].dt.day_name()
df['Day']=df['Date/Time'].dt.day
df['Month']=df['Date/Time'].dt.month
df['Minute']=df['Date/Time'].dt.minute
df['Hour']=df['Date/Time'].dt.hour

In [ ]:
df.head()

## Analysis of trips by weekdays

In [ ]:
!pip install plotly
import plotly.express as px

In [ ]:
df['Weekday'].value_counts()

In [ ]:
Title= 'Analysis of trips by weekdays'
fig=px.bar(df,x=df['Weekday'].value_counts().index,
       y=df['Weekday'].value_counts() ,
       labels={'x': 'Days Of The Week', 'y':'Number Of Rides'},
        title=Title,
        color=["red", "goldenrod", "#00D","green","blue","yellow","brown"],
        height=600,
      )
fig.show()

In [ ]:
df["Hour"].unique()

In [ ]:
df['Month'].unique()

In [ ]:
df.dtypes

# Analysis of trips per Hour for every month

In [ ]:
#analysis of trips for every hour for all months
#plt.hist(df["Hour"], label='Hour');
#plt.title('Trips per hour');
plt.figure(figsize=(20,10))
ax=sns.countplot(df['Hour'])
ax.set_title('Trips per Hour 2014', fontsize = 18)
ax.set_ylabel('Number Of Rides', fontsize = 15)
ax.set_xlabel ('Hour', fontsize = 15)

## Analysis of trips for every hour per month

In [ ]:
plt.figure(figsize=(20,15))
for i,month in enumerate(df["Month"].unique()):
    plt.subplot(3,2,i+1)
    x= df[df["Month"]==month]["Hour"]
    x.hist(
            color ='pink'
          
           )
    plt.xlabel('Hour H')
    plt.ylabel('Number of Rides at Hour H')
    plt.title('Number Of rides per hour for the month : '+str(month),
          fontweight ="bold")



## Which month has the most trips?

In [ ]:
df.head()

In [ ]:
!pip install chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
df.groupby('Month')['Hour'].count()

In [ ]:
    trace1=go.Bar(
          x=df.groupby('Month')['Hour'].count().index,
          y=df.groupby('Month')['Hour'].count(),
          marker_color=[px.colors.qualitative.Dark24[i+1],px.colors.qualitative.Dark24[i+2],px.colors.qualitative.Dark24[i+3],px.colors.qualitative.Dark24[i+4],px.colors.qualitative.Dark24[i+5],px.colors.qualitative.Dark24[i+6]],
        text='Total trips per month'
    )
    iplot([trace1])


      


## Which day of the month has the most trips?

In [ ]:
plt.figure(figsize=(20,10));
plt.hist(df["Day"],bins=30, rwidth=0.8,range=(0,30.5));
plt.xlabel('Day Of The Month')
plt.ylabel('Trips Per Day')
plt.title('Total Trips By The Month Day');

## Analysis of total rides month wise


In [ ]:
plt.figure(figsize=(20,15))
for i,month in enumerate (df['Month'].unique(),1):
    plt.subplot(3,2,i)
    df_new=df[df["Month"]==month]
    plt.hist(df_new['Day'],  color ='skyblue')
    plt.xlabel('Days in month: {}'.format(month))
    plt.ylabel('Total Trips')
    plt.title('Total rides for month {} /day of the month'.format(month))

## Analysis of Rush for each hour

In [ ]:
plt.figure(figsize=(20,15))
ax=sns.pointplot(x='Hour', y='Lat', data=df, hue='Weekday')
ax.set_title("HoursofDay of  VS LatitudeofPassengers")

## Analyzing which base number gets popular by month name

In [ ]:
df.head()

In [ ]:
base=df.groupby(["Base","Month"])['Date/Time'].count().reset_index()
base

In [ ]:
plt.figure(figsize=(20,15));
#sns.lineplot(x='Month',y='Date/Time', hue='Base', data=base);
px.line(base, x="Month", y="Date/Time", title='Evolution of base numbers by month', color='Base')


## Cross Analysis with Heatmaps

In [ ]:
def count_rows(rows):
    return len(rows)

### Heatmap by Hour and Weekday

In [ ]:
by_cross=df.groupby(['Weekday','Hour']).apply(count_rows)
by_cross.head()

In [ ]:
pivot=by_cross.unstack()
pivot.head()

In [ ]:
plt.figure(figsize=(20,15));
px.imshow(pivot,
          labels=dict(color="Number Of Trips"),
          title=' Heatmap by Hour and Weekday'
         )


### Automation of Heatmaps

In [ ]:
def heatmap(col1,col2):
    by_cross=df.groupby([col1,col2]).apply(count_rows)
    pivot=by_cross.unstack()
    plt.figure(figsize=(20,15));
    return px.imshow(pivot,
          labels=dict(color="Number Of Trips"),
          title=' Heatmap by {} and {}'.format(col1,col2)
         )


In [ ]:
heatmap('Month','Hour')

In [ ]:
heatmap('Month','Weekday')

## Analysis of location data points

In [ ]:
df.head()

In [ ]:
plt.plot(df['Lon'],df['Lat'],'r+')

In [ ]:
fig = px.line(df, x="Lat", y="Lon")
fig.show()

In [ ]:
df.head()

## Spatial analysis using heatmap to get a clear cut of Rush

In [ ]:
sun=df[df['Weekday']=='Sunday' ]
sun.head()

In [ ]:
rush_sun=sun.groupby(['Lat','Lon'])['Weekday'].count().reset_index()
rush_sun.columns=["Lat",'Lon','No of trips']
rush_sun.head()

In [ ]:
!pip install folium

In [ ]:
from folium.plugins import HeatMap

In [ ]:
import folium

In [ ]:
basemap=folium.Map()
HeatMap(rush_sun,zoom=20, radius=30).add_to(basemap)
basemap

## Automating the analysis

In [ ]:
def plot(df,day):
    basemap=folium.Map()
    x=df[df['Weekday']==day ]
    HeatMap(x.groupby(['Lat','Lon'])['Weekday'].count().reset_index(), zoom=30,radius=30).add_to(basemap)
    return basemap

# Data analysis for 2015( Jan-July)

In [ ]:
df_uber = pd.read_csv('D:\/uber_nyc_data.csv')
df_uber.head()

In [ ]:
df_uber.info()
#1.4 GB + 31 million entries

## Check for missing or possibly erroneous data

In [ ]:
len(df_uber[df_uber.duplicated() == True])
# ==> No duplicated rows

In [ ]:
df_uber.isnull().any()
# ==> destination_taz+trip_distance+trip_duration have null values
df_uber.isnull().sum()


In [ ]:
origin=df_uber["origin_taz"].unique()
print(len(origin)) ## we have 28 unique origin codes
origin

In [ ]:
dest=df_uber["destination_taz"].unique()
print(len(dest))
dest
## we have 30 destination

In [ ]:
set(dest) - set(origin) ## we have 2 diff 18 and a null value

In [ ]:
#Check data with null destinations
df_uber[df_uber["destination_taz"].isnull()].head()
#==>Solution: predict dest based on other features

In [ ]:
#Check data with null trip_distance && trip_duration
df38=df_uber[df_uber["trip_distance"].isnull() & df_uber["trip_duration"].isnull()]
df38.head()
#Nor data is missing the destination

In [ ]:
len(df_uber[(df_uber["trip_distance"].isnull()== False) & (df_uber["trip_duration"].isnull()== True)])
#All cases corresponding to missing values in trip_distance correspond to trip_duration

In [ ]:
df_uber.info()
#==> pickup_datetime type is an object

## Parse the pickup_datetime column to datetime type

In [ ]:
len(df_uber[df_uber["pickup_datetime"].apply(lambda x: x[-5:] =='00:00')== False])
#None of the pickup_datetime offer minutes or seconds precision

In [ ]:
def dateparser(s):
    return datetime.datetime(int(s[0:4]), int(s[5:7]), int(s[8:10]), int(s[11:13]))

In [ ]:
## Apply parser
import datetime
df_uber['y_d_h']=df_uber["pickup_datetime"].apply(dateparser)

In [ ]:
beginning_date=df_uber["y_d_h"].min()
ending_date=df_uber["y_d_h"].max()
print (beginning_date, ending_date, ending_date-beginning_date)
##==> data from 1 september 2014 to 1 september 2015

In [ ]:
df_uber.head()

In [ ]:
df_uber.drop(columns='pickup_datetime',inplace=True)

In [ ]:
df_uber.head()

## Get the federal holidays for the period (useful for analyzing data later)

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
holidays=USFederalHolidayCalendar().holidays(beginning_date,ending_date, return_name=True)
holidays

In [ ]:
#Extract the weekday for each holiday
##The strftime() method returns a string representing date and time using date, time or datetime object.
holidays.index.map(lambda x: x.strftime('%a'))
